In [5]:
import numpy as np
from scipy.signal import welch
from reproduce import *
# -------------------------------------------------------------
# Single-trace reproduction of Li et al. sustained oscillation
# -------------------------------------------------------------

# @title Helper Functions

def default_pars(**kwargs):
  pars = {}

  # Excitatory parameters
  pars['tau_E'] = 1.     # Timescale of the E population [ms]
  pars['a_E'] = 1.2      # Gain of the E population
  pars['theta_E'] = 2.8  # Threshold of the E population

  # Inhibitory parameters
  pars['tau_I'] = 2.0    # Timescale of the I population [ms]
  pars['a_I'] = 1.0      # Gain of the I population
  pars['theta_I'] = 4.0  # Threshold of the I population

  # Connection strength
  pars['wEE'] = 9.   # E to E
  pars['wEI'] = 4.   # I to E
  pars['wIE'] = 13.  # E to I
  pars['wII'] = 11.  # I to I

  # External input
  pars['I_ext_E'] = 0.
  pars['I_ext_I'] = 0.

  # simulation parameters
  pars['T'] = 50.        # Total duration of simulation [ms]
  pars['dt'] = .1        # Simulation time step [ms]
  pars['rE_init'] = 0.2  # Initial value of E
  pars['rI_init'] = 0.2  # Initial value of I

  # External parameters if any
  for k in kwargs:
      pars[k] = kwargs[k]

  # Vector of discretized time points [ms]
  pars['range_t'] = np.arange(0, pars['T'], pars['dt'])

  return pars


def simulate_wc(tau_E, a_E, theta_E, tau_I, a_I, theta_I,
                wEE, wEI, wIE, wII, I_ext_E, I_ext_I,
                rE_init, rI_init, dt, range_t, **other_pars):
  """
  Simulate the Wilson-Cowan equations

  Args:
    Parameters of the Wilson-Cowan model

  Returns:
    rE, rI (arrays) : Activity of excitatory and inhibitory populations
  """
  # Initialize activity arrays
  Lt = range_t.size
  rE = np.append(rE_init, np.zeros(Lt - 1))
  rI = np.append(rI_init, np.zeros(Lt - 1))
  I_ext_E = I_ext_E * np.ones(Lt)
  I_ext_I = I_ext_I * np.ones(Lt)

  # Simulate the Wilson-Cowan equations
  for k in range(Lt - 1):

    # Calculate the derivative of the E population
    drE = dt / tau_E * (-rE[k] + F(wEE * rE[k] - wEI * rI[k] + I_ext_E[k],
                                   a_E, theta_E))

    # Calculate the derivative of the I population
    drI = dt / tau_I * (-rI[k] + F(wIE * rE[k] - wII * rI[k] + I_ext_I[k],
                                   a_I, theta_I))

    # Update using Euler's method
    rE[k + 1] = rE[k] + drE
    rI[k + 1] = rI[k] + drI

  return rE, rI

  
pars_osc = default_pars(
    tau_E=20., tau_I=10.,
    a_E=1., a_I=1.,
    theta_E=5., theta_I=20.,
    wEI=26., wIE=20.,
    wEE=25., wII=0.,       # <- IN BEAN
    I_ext_E=2., I_ext_I=7.,
    T=1000, dt=0.05,       # 1-second trace, 0.05-ms step
)

rE, rI = simulate_wc(**pars_osc)

# --- plot the time-series ---
import matplotlib.pyplot as plt
t = pars_osc['range_t']
plt.figure(figsize=(7,3))
plt.plot(t, rE, label='E', color='b')
plt.plot(t, rI, label='I', color='r', alpha=.7)
plt.xlim(0, 600)           # show first 600 ms
plt.xlabel('Time (ms)')
plt.ylabel('Activity')
plt.title('Sustained oscillation (Li-style gamma trace)')
plt.legend(); plt.show()

# --- quick frequency check ---
sig = rE_tail - rE_tail.mean()
fs  = 1000 / pars['dt']              # <-- use milliseconds ➜ seconds
f, P = welch(sig, fs, nperseg=2048)
peak = np.argmax(P[1:]) + 1          # skip the DC bin
f_peak = f[peak]

print(f'Peak frequency ≈ {f_peak:.1f} Hz')







NameError: name 'F' is not defined